In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.model_selection import train_test_split
!pip install datasets
!pip install evaluate
from datasets import Dataset
import evaluate

##
from google.colab import drive
drive.mount('/content/drive')
##
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#event vectors
event_traces = pd.read_csv('/content/drive/MyDrive/Data/Event_traces.csv',usecols=['BlockId','Label','Features'])
event_traces['Label'] = event_traces['Label'].map({'Success':0,'Fail':1})
#event template- message
log_templates = pd.read_csv('/content/drive/MyDrive/Data/HDFS.log_templates.csv')
#Printing-Displaying
display(HTML((event_traces.sample(5)).to_html()))
#
display(HTML((log_templates.sample(5)).to_html()))
##Dictionary ton EventIds-Events
event_dictionary = dict(zip(log_templates['EventId'],log_templates['EventTemplate']))

print(event_dictionary)

#Information
event_traces.info()
print(event_traces.shape)
print(event_traces.describe())
#
#input('WAIT')
##
def features_to_strings(entry):
  return " ".join([event_dictionary.get(eventID) for eventID in entry['Features'].replace('[','').replace(']','').split(',')])

###
##
print('Class distribution: ',event_traces['Label'].value_counts())
#
display(HTML((event_traces.sample(1)).to_html()))
#
#input('WAIT')
event_traces.drop(columns=['BlockId'],inplace=True) #drop the block id
#drop a portion of 0's labels as the data is too big and to lower the frequency of this class
#
event_traces_0 = event_traces[event_traces['Label']==0]
event_traces_1 = event_traces[event_traces['Label']==1]
event_traces_0 = event_traces_0.sample(frac=0.05,random_state=42)#an valo 0.035-0.3 eimai poly konta sta #arithmo klaseon 1
event_traces = pd.concat([event_traces_0,event_traces_1],ignore_index=True)
event_traces['Features'] = event_traces.apply(features_to_strings,axis=1)

#
print('Class distribution: ',event_traces['Label'].value_counts())
#input('WAIT')
#Train test split#
event_traces_train,event_traces_test = train_test_split(event_traces,test_size=0.2,random_state=42,shuffle=True)
display(HTML((event_traces_train.sample(1)).to_html()))
#
event_traces_train = Dataset.from_pandas(event_traces_train)
event_traces_test = Dataset.from_pandas(event_traces_test)

input('WAIT')
##
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # tokenizer
#orisma ena line tou dataset opote epilego to column me to keimeno pou thelo na tokenaro
def tokenize_logs(entry):
  tokens = tokenizer(entry['Features'],padding='max_length',truncation=True)
  tokens['labels'] = entry['Label']
  return tokens

#truncation true an einai parapano apo to megethos tis paragrafou pou borei na xeiristei to modelo thn kanei truncate mexri to max length
#
'''
'''
#Tokenizing#
#event_traces_train['Features'] = event_traces_train.apply(tokenize_logs,axis=1)
#event_traces_test['Features'] = event_traces_test.apply(tokenize_logs,axis=1)
event_traces_train = event_traces_train.map(tokenize_logs,batched=True)
event_traces_test = event_traces_test.map(tokenize_logs,batched=True)
#
print(event_traces_train[0])
#
#
#display(HTML((event_traces_train.sample(1)).to_html()))
input('WAIT')
#########Fine tuning meso ton parakato parametron##########
###METRICS####

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

###MODEL###
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2).to(device)#modelo me ena extra layer gia to classification
###
training_arguments = TrainingArguments(
    output_dir = "./BERTOnLogs",#pou tha apothikeytei to modelo
    eval_strategy = "epoch",#to evaluation ginetai sto telos kathe epoch
    learning_rate=2e-5,#rythmos mathisis genika na einai mikros xoris akrotites. mikrh timh-> kalyterh genikeysh
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,#genika de thelei oute poly liga alla oute polla.poses fores to modelo tha iterarei to training dataset-koinos se posa epochs tha ginei to training
    weight_decay=0.01#meiosh tou overfitting
)

#########
trainer = Trainer(
    model=model,#modelo pou tha kanei to classification,
    args=training_arguments,#ta orismata tis ekpaideyshs apo pano
    #tokenizer=tokenizer#o tokenizer, an exo kanei hdh preprocessing-tokenizing LOGIKA den to xreiazomai
    train_dataset= event_traces_train,#to synolo ekpaideyshs gia to fine-tuning
    eval_dataset=event_traces_test,#to synolo elegxou-aksiologisis gia to fine tuning-xoris ayto de tha aksiologithei h apodosi tou modelou mono tha ekpaideytei
    #tokenizer=tokenizer
    compute_metrics=compute_metrics#exei os apotelesma sto training tou modelou ektos apo to training/validation loss na emfanizontai kai oi times ton metrikon pou orisa sth synartisi kai epistrefontai apo ayth
    )



### Train the model
trainer.train()
#results = trainer.evaluate() #ti apodosi fernei to modelo sto test dataset


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
Mounted at /content/drive
cpu


,BlockId,Label,Features
432720,blk_8206426068889797526,1,"[E5,E22]"
92832,blk_-6027304217285375527,0,"[E22,E5,E5,E5,E26,E26,E26,E11,E9,E11,E9,E11,E9,E3,E3,E4,E3,E4,E3,E3,E3,E4,E23,E23,E23,E21,E21,E21]"
292612,blk_4071644970492546692,0,"[E5,E5,E5,E22,E11,E9,E11,E9,E26,E26,E11,E9,E26,E23,E23,E23,E21,E21,E21]"
73246,blk_691757021928729693,0,"[E22,E5,E5,E5,E26,E26,E26,E11,E9,E11,E9,E11,E9,E3,E4,E3,E3,E3,E4,E3,E3,E4,E3,E3,E4,E23,E23,E23,E21,E21,E21]"
21930,blk_7484623947600160260,0,"[E5,E5,E5,E22,E11,E9,E11,E9,E26,E26,E11,E9,E26,E4,E3,E4,E23,E23,E23,E21,E21,E21]"


,EventId,EventTemplate
12,E13,[*]Receiving empty packet for block[*]
1,E2,[*]Verification succeeded for[*]
6,E7,[*]writeBlock[*]received exception[*]
13,E14,[*]Exception in receiveBlock for block[*]
23,E24,[*]BLOCK* Removing block[*]from neededReplications as it does not belong to any file[*]


{'E1': '[*]Adding an already existing block[*]', 'E2': '[*]Verification succeeded for[*]', 'E3': '[*]Served block[*]to[*]', 'E4': '[*]Got exception while serving[*]to[*]', 'E5': '[*]Receiving block[*]src:[*]dest:[*]', 'E6': '[*]Received block[*]src:[*]dest:[*]of size[*]', 'E7': '[*]writeBlock[*]received exception[*]', 'E8': '[*]PacketResponder[*]for block[*]Interrupted[*]', 'E9': '[*]Received block[*]of size[*]from[*]', 'E10': '[*]PacketResponder[*]Exception[*]', 'E11': '[*]PacketResponder[*]for block[*]terminating[*]', 'E12': '[*]:Exception writing block[*]to mirror[*]', 'E13': '[*]Receiving empty packet for block[*]', 'E14': '[*]Exception in receiveBlock for block[*]', 'E15': '[*]Changing block file offset of block[*]from[*]to[*]meta file offset to[*]', 'E16': '[*]:Transmitted block[*]to[*]', 'E17': '[*]:Failed to transfer[*]to[*]got[*]', 'E18': '[*]Starting thread to transfer block[*]to[*]', 'E19': '[*]Reopen Block[*]', 'E20': '[*]Unexpected error trying to delete block[*]BlockInfo 

,BlockId,Label,Features
379759,blk_-7646970982977995594,0,"[E5,E22,E5,E5,E11,E9,E11,E9,E11,E9,E26,E26,E26,E23,E23,E23,E21,E21,E21]"


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
